In [1]:
import numpy as np
import pandas as pd

In [2]:
class Node():
    def __init__(self, feature_index = None, threshold = None, left=None, right=None, information_gain = None, value = None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.information_gain = information_gain
        self.value = value


In [3]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split = 2, max_depth = 2):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, current_depth = 0):
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        if num_samples >= self.min_samples_split and current_depth <= self.max_depth:
            # find the best split and assign left and right nodes
            
            best_split = self.find_best_split(dataset)
            if best_split['information_gain'] > 0:
                left  = self.build_tree(best_split['left_dataset'], current_depth + 1)
                right = self.build_tree(best_split['right_dataset'], current_depth + 1)
                                        
                return Node(feature_index = best_split['feature_index'],
                            threshold = best_split['threshold'],
                            left = left,
                            right = right,
                            information_gain = best_split['information_gain']
                            )
            
        
        # find the value of the node and return a lead Node
        value = self.calculateLeafValue(Y)
        print("Reached a leaf...")
        return Node(value = value)
        
    def find_best_split(self, dataset):
        best_split = {}
        max_information_gain = -float("inf")
        num_samples, num_features = np.shape(dataset[:, :-1])
        
        for feature_index in range(num_features):
            feature_values = np.unique(X[:, feature_index])
            for threshold in feature_values:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    split_information_gain = self.calculateInformationGain(y, left_y, right_y, 'gini')
                    if split_information_gain > max_information_gain:
                        max_information_gain = split_information_gain
                        best_split['feature_index'] = feature_index
                        best_split['information_gain'] = split_information_gain
                        best_split['threshold'] = threshold
                        best_split['left_dataset'] = dataset_left
                        best_split['right_dataset'] = dataset_right
        
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        dataset_left = []
        dataset_right = []
        for row in dataset:
            if row[feature_index] <= threshold:
                dataset_left.append(row)
            else:
                dataset_right.append(row)
        
        return np.array(dataset_left), np.array(dataset_right)
    
    def calculateInformationGain(self, parent, left_child, right_child, mode = 'entropy'):
        weight_left = len(left_child) / len(parent)
        weight_right = len(right_child) / len(parent)
        if mode == 'entropy':
            return self.entropy(parent) - weight_left * self.entropy(left_child) - weight_right * self.entropy(right_child)
        if mode == 'gini':
            return self.gini(parent) - weight_left * self.gini(left_child) - weight_right * self.gini(right_child)
                        
    def entrop(self, y):
        classes = np.unique(y)
        entropy = 0
        for cls in classes:
            prob = len(y[y == cls]) / len(y)
            entropy += -prob * np.log2(prob)
        return entropy
    
    def gini(self, y):
        classes = np.unique(y)
        gini = 1
        for cls in classes:
            prob = len(y[y == cls]) / len(y)
            gini += - prob * prob
        return gini
                
    def calculateLeafValue(self, y):
        y = list(y)
        return max(y, key = y.count)
    
    def fit(self, X, Y):
        dataset = np.concatenate((X, Y), axis = 1)
        self.root = self.build_tree(dataset)
        
    def printTree(self, tree = None, indent = " "):
        if not tree:
            tree = self.root
        
        if tree.value is not None:
            print(tree.value)
        else:
            print("X_" + str(tree.feature_index) + " <= " + str(tree.threshold) + " ? " + str(tree.information_gain))
            print("%sleft:" % (indent), end = "")
            self.printTree(tree.left, indent + indent)
            print("%sright:" % (indent), end = "")
            self.printTree(tree.right, indent + indent)
            
    def predict(self, X):
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions
    
    def make_prediction(self, x, tree):
        if tree.value != None:
            return tree.value
        if x[tree.feature_index] <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
        
            
        

In [4]:
col_names = ["sepal_length", "sepal_width", "petal_length", "petal_width", "type"]
data = pd.read_csv('./Data/iris.csv', skiprows = 1, header = None, names = col_names)

data.head()

,sepal_length,sepal_width,petal_length,petal_width,type
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
types = np.unique(data['type'])
print(types)
data['type'].replace(types, range(len(types)), inplace = True)
data.head(10)

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


,sepal_length,sepal_width,petal_length,petal_width,type
1,5.1,3.5,1.4,0.2,0
2,4.9,3.0,1.4,0.2,0
3,4.7,3.2,1.3,0.2,0
4,4.6,3.1,1.5,0.2,0
5,5.0,3.6,1.4,0.2,0
6,5.4,3.9,1.7,0.4,0
7,4.6,3.4,1.4,0.3,0
8,5.0,3.4,1.5,0.2,0
9,4.4,2.9,1.4,0.2,0
10,4.9,3.1,1.5,0.1,0


In [6]:
X = data.iloc[:,:-1].values
Y = data.iloc[:,-1].values.reshape(-1, 1)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 41)

In [8]:
myClassifier = DecisionTreeClassifier(min_samples_split = 3, max_depth = 3)
myClassifier.fit(X_train, Y_train)

Reached a leaf...
Reached a leaf...
Reached a leaf...
Reached a leaf...
Reached a leaf...
Reached a leaf...


In [9]:
myClassifier.printTree()

X_2 <= 1.9 ? 0.33741385372714494
 left:0.0
 right:X_3 <= 1.5 ? 0.427106638180289
  left:X_2 <= 4.9 ? 0.051246537396121776
    left:1.0
    right:2.0
  right:X_2 <= 5.0 ? 0.019631171921475288
    left:X_1 <= 2.8 ? 0.20833333333333331
        left:2.0
        right:1.0
    right:2.0


In [10]:
y_pred = myClassifier.predict(X_test)

In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, y_pred)

0.9333333333333333